In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

from torchvision import transforms, datasets
import numpy as np
from matplotlib import pyplot as plt
import time
import sys
import os
import h5py

# loading the folds:
with open('../../data/stl10_binary/fold_indices.txt', 'r') as f_folds:
    folds = f_folds.readlines()

k_folds = []
for fold_i in folds:
    k_folds.append(np.asarray(fold_i.split(' ')[:-1], dtype=int))

print "Folds loaded:"
print len(folds), type(folds), type(folds[0]), len(folds[0])
print len(k_folds[0]), type(k_folds[0][0]), k_folds[0][0]

Folds loaded:
10 <type 'list'> <type 'str'> 4773
1000 <type 'numpy.int64'> 39


/home/eric/.local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# read data from hdf5
exp_numb = '086'
f_train = h5py.File('./' + exp_numb + '_L2_long_2_train.hdf5', 'r')
f_val = h5py.File('./' + exp_numb + '_L2_long_2_val.hdf5', 'r')
#f_train = h5py.File('temp_18304_train.hdf5', 'r')
#f_val = h5py.File('temp_18304_val.hdf5', 'r')

X_train = f_train['data']
y_train = f_train['labels']

X_val = f_val['data']
y_val = f_val['labels']

y_val = np.ravel(y_val)
y_train = np.ravel(y_train)

In [3]:
X_train.shape

(5000, 71536)

In [4]:
from sklearn import svm
C_SVM =[10, 40, 80, 100, 120]
#[0.1, 2, 5, 10]
#[10, 40, 70, 90]
#[0.01, 0.1, 1, 10, 20, 30, 40]
#[1e-08, 5e-8, 1e-7, 5e-7, 1e-6, 5e-6, 1e-5]
#[0.01, 0.1, 1, 10, 20, 30, 40]
#[1e-7, 5e-7,1e-6, 5e-6, 1e-5]
#[0.01, 0.1, 1, 10, 20, 30, 40]
#[1e-08, 1e-07, 1e-06, 1e-05, 1e-4]
#[4e-08, 6e-08, 8e-08, 1e-07, 4e-07]

# [1e-3, 1e-2, 1e-01, 1, 10] ### 20K and L2
#[2e-7, 2e-6, 2e-05, 2e-04] ### 20K features NO L2
#[1e-6, 5e-6, 1e-05, 5e-05]
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(2*np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(2*np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 10 		 Time:1567.9
Training for C = 40 		 Time:3709.12
Training for C = 80 		 Time:4435.68
Training for C = 100 		 Time:4360.06
Training for C = 120 		 Time:4326.63
 
Train 	 Validation 	 Validation var 	 C value
93.24 	   74.35 	    1.1308 		  10
99.54 	   76.2 	    1.0328 		  40
99.98 	   76.41 	    0.6924 		  80
100.0 	   76.42 	    0.694 		  100
100.0 	   76.41 	    0.7267 		  120
 
Train 	 Validation 	 Validation var
99.92 	   76.5 	    0.7406

Values for C used: 
[40, 80, 120, 80, 120, 100, 120, 100, 120, 40]

Set of the values for C used: 
set([40, 80, 100, 120])

First and last value used: 
40 120


evaluating...

In [ ]:
from sklearn import svm
C_SVM =[150, 200, 250, 300, 350, 400]
#[0.1, 2, 5, 10]
#[10, 40, 70, 90]
#[0.01, 0.1, 1, 10, 20, 30, 40]
#[1e-08, 5e-8, 1e-7, 5e-7, 1e-6, 5e-6, 1e-5]
#[0.01, 0.1, 1, 10, 20, 30, 40]
#[1e-7, 5e-7,1e-6, 5e-6, 1e-5]
#[0.01, 0.1, 1, 10, 20, 30, 40]
#[1e-08, 1e-07, 1e-06, 1e-05, 1e-4]
#[4e-08, 6e-08, 8e-08, 1e-07, 4e-07]

# [1e-3, 1e-2, 1e-01, 1, 10] ### 20K and L2
#[2e-7, 2e-6, 2e-05, 2e-04] ### 20K features NO L2
#[1e-6, 5e-6, 1e-05, 5e-05]
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(2*np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(2*np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 150

evaluating...

In [19]:
from sklearn import svm
C_SVM =[1000, 1500, 2000,2500]
#[0.1, 2, 5, 10]
#[10, 40, 70, 90]
#[0.01, 0.1, 1, 10, 20, 30, 40]
#[1e-08, 5e-8, 1e-7, 5e-7, 1e-6, 5e-6, 1e-5]
#[0.01, 0.1, 1, 10, 20, 30, 40]
#[1e-7, 5e-7,1e-6, 5e-6, 1e-5]
#[0.01, 0.1, 1, 10, 20, 30, 40]
#[1e-08, 1e-07, 1e-06, 1e-05, 1e-4]
#[4e-08, 6e-08, 8e-08, 1e-07, 4e-07]

# [1e-3, 1e-2, 1e-01, 1, 10] ### 20K and L2
#[2e-7, 2e-6, 2e-05, 2e-04] ### 20K features NO L2
#[1e-6, 5e-6, 1e-05, 5e-05]
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(2*np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(2*np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 1000 		 Time:142.06
Training for C = 1500 		 Time:140.19
Training for C = 2000 		 Time:139.48
Training for C = 2500 		 Time:142.22
 
Train 	 Validation 	 Validation var 	 C value
99.87 	   69.19 	    1.2538 		  1000
99.82 	   68.25 	    1.851 		  1500
99.79 	   68.23 	    1.4419 		  2000
99.96 	   68.72 	    1.5412 		  2500
 
Train 	 Validation 	 Validation var
99.87 	   69.32 	    1.078

Values for C used: 
[1000, 1500, 1000, 2500, 2000, 1000, 1000, 1000, 2500, 1000]

Set of the values for C used: 
set([1000, 2500, 1500, 2000])

First and last value used: 
1000 2500


evaluating...

In [10]:
from sklearn import svm
C_SVM = [20, 40, 60, 80, 100]

#C_SVM = [100, 120, 140, 160, 180, 200]


c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(2*np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(2*np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 20 		 Time:797.2
Training for C = 40 		 Time:1054.24
Training for C = 60 		 Time:1227.71
Training for C = 80 		 Time:1342.52
Training for C = 100 		 Time:1395.06
 
Train 	 Validation 	 Validation var 	 C value
100.0 	   69.92 	    1.1544 		  20
100.0 	   69.68 	    1.2059 		  40
100.0 	   69.57 	    1.1542 		  60
100.0 	   69.49 	    1.1503 		  80
100.0 	   69.44 	    1.1844 		  100
 
Train 	 Validation 	 Validation var
100.0 	   69.92 	    1.1544

Values for C used: 
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]

Set of the values for C used: 
set([20])

First and last value used: 
20 20


evaluating...

In [13]:
from sklearn import svm
C_SVM = [0.1, 1, 3, 6, 8, 10]#, 90, 100]
#[8e-8, 1e-7, 2e-7, 3e-7, 4e-7, 5e-7]
#[40, 50, 60, 70, 80, 90, 100]

#[1e-8, 5e-8, 1e-7]

#[50, 60]#, 80, 100]

# [1e-3, 1e-2, 1e-01, 1, 10] ### 20K and L2
# [5e-8, 1e-7, 5e-07] ### 20K features no L2
# [2e-7, 2e-6, 2e-05, 2e-04]
#[1e-6, 5e-6, 1e-05, 5e-05]
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(2*np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(2*np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 0.1 		 Time:9.8
Training for C = 1 		 Time:17.29
Training for C = 3 		 Time:27.36
Training for C = 6 		 Time:38.2
Training for C = 8 		 Time:43.52
Training for C = 10 		 Time:49.83
 
Train 	 Validation 	 Validation var 	 C value
73.46 	   60.73 	    0.7103 		  0.1
91.08 	   68.01 	    0.5808 		  1
97.96 	   69.27 	    0.7824 		  3
99.72 	   69.34 	    0.8728 		  6
99.89 	   69.26 	    0.8672 		  8
99.97 	   69.19 	    0.9931 		  10
 
Train 	 Validation 	 Validation var
98.86 	   69.4 	    0.8263

Values for C used: 
[3, 3, 6, 6, 3, 8, 3, 3, 6, 6]

Set of the values for C used: 
set([8, 3, 6])

First and last value used: 
3 8


evaluating...

In [7]:
from sklearn import svm
C_SVM = [1,4,7,10]

#[100, 110, 150, 170, 200]#, 80, 100]

# [1e-3, 1e-2, 1e-01, 1, 10] ### 20K and L2
# [5e-8, 1e-7, 5e-07] ### 20K features no L2
# [2e-7, 2e-6, 2e-05, 2e-04]
#[1e-6, 5e-6, 1e-05, 5e-05]
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(2*np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(2*np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 1 		 Time:59.92
Training for C = 4 		 Time:105.73
Training for C = 7 		 Time:134.38
Training for C = 10 		 Time:153.27
 
Train 	 Validation 	 Validation var 	 C value
94.1 	   67.9 	    1.0271 		  1
99.75 	   68.34 	    0.949 		  4
99.99 	   68.04 	    1.027 		  7
100.0 	   67.84 	    1.0828 		  10
 
Train 	 Validation 	 Validation var
99.75 	   68.34 	    0.949

Values for C used: 
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

Set of the values for C used: 
set([4])

First and last value used: 
4 4


evaluationg...

In [8]:
from sklearn import svm
C_SVM = [2,3,4,5,6]#, 80, 100]

# [1e-3, 1e-2, 1e-01, 1, 10] ### 20K and L2
# [5e-8, 1e-7, 5e-07] ### 20K features no L2
# [2e-7, 2e-6, 2e-05, 2e-04]
#[1e-6, 5e-6, 1e-05, 5e-05]
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(2*np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(2*np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 2 		 Time:78.74
Training for C = 3 		 Time:95.48
Training for C = 4 		 Time:107.02
Training for C = 5 		 Time:117.42
Training for C = 6 		 Time:126.85
 
Train 	 Validation 	 Validation var 	 C value
98.15 	   68.31 	    0.901 		  2
99.37 	   68.38 	    0.9656 		  3
99.75 	   68.33 	    0.946 		  4
99.92 	   68.27 	    0.885 		  5
99.95 	   68.17 	    0.9774 		  6
 
Train 	 Validation 	 Validation var
99.25 	   68.44 	    0.9574

Values for C used: 
[2, 3, 5, 3, 5, 3, 2, 3, 3, 3]

Set of the values for C used: 
set([2, 3, 5])

First and last value used: 
2 5


In [12]:
from sklearn import svm
C_SVM = [20]

# [1e-3, 1e-2, 1e-01, 1, 10] ### 20K and L2
# [5e-8, 1e-7, 5e-07] ### 20K features no L2
# [2e-7, 2e-6, 2e-05, 2e-04]
#[1e-6, 5e-6, 1e-05, 5e-05]
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(2*np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(2*np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 20 		 Time:402.9
 
Train 	 Validation 	 Validation var 	 C value
99.04 	   70.27 	    1.1109 		  20
 
Train 	 Validation 	 Validation var
99.04 	   70.27 	    1.1109

Values for C used: 
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]

Set of the values for C used: 
set([20])

First and last value used: 
20 20


In [13]:
from sklearn import svm
C_SVM = [15]

# [1e-3, 1e-2, 1e-01, 1, 10] ### 20K and L2
# [5e-8, 1e-7, 5e-07] ### 20K features no L2
# [2e-7, 2e-6, 2e-05, 2e-04]
#[1e-6, 5e-6, 1e-05, 5e-05]
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(2*np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(2*np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 15 		 Time:357.97
 
Train 	 Validation 	 Validation var 	 C value
98.14 	   70.2 	    1.0536 		  15
 
Train 	 Validation 	 Validation var
98.14 	   70.2 	    1.0536

Values for C used: 
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]

Set of the values for C used: 
set([15])

First and last value used: 
15 15


In [4]:
from sklearn import svm
C_SVM = [0.01,0.1,1,10,20]
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 0.01 		 Time:133.82
Training for C = 0.1 		 Time:174.32
Training for C = 1 		 Time:408.65
Training for C = 10 		 Time:1651.3
Training for C = 20 		 Time:2765.96
 
Train 	 Validation 	 Validation var 	 C value
57.56 	   52.28 	    0.7884 		  0.01
62.1 	   55.82 	    0.7767 		  0.1
72.5 	   63.73 	    0.4778 		  1
88.45 	   70.72 	    0.3405 		  10
93.92 	   72.06 	    0.3155 		  20
 
Train 	 Validation 	 Validation var
93.92 	   72.06 	    0.3155

Values for C used: 
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20]

Set of the values for C used: 
set([20])

First and last value used: 
20 20


In [ ]:
# 69_ep110 with l2

In [5]:
from sklearn import svm
C_SVM = [25, 50, 75, 100] 
#[2e-8, 2e-7, 2e-6, 2e-5]
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(np.std(val_max),6)*100
print ''
print "Values for C used: "
values_C = [C_SVM[idx] for idx in train_max_arg]
values_C_set = set(values_C)
print values_C
print ''
print "Set of the values for C used: "
print values_C_set
print ''
print "First and last value used: "
values_C.sort()
print values_C[0],  values_C[-1]

Training for C = 25 		 Time:3141.95
Training for C = 50 		 Time:4602.37
Training for C = 75 		 Time:4743.37
Training for C = 100 		 Time:4619.39
 
Train 	 Validation 	 Validation var 	 C value
95.68 	   72.38 	    0.3221 		  25
98.88 	   72.93 	    0.2839 		  50
99.7 	   72.95 	    0.4001 		  75
99.88 	   72.93 	    0.4414 		  100
 
Train 	 Validation 	 Validation var
99.37 	   73.05 	    0.3517

Values for C used: 
[100, 50, 50, 75, 50, 100, 50, 75, 100, 75]

Set of the values for C used: 
set([50, 75, 100])

First and last value used: 
50 100


In [4]:
from sklearn import svm
C_SVM = np.linspace(1e-6, 2e-5, 10)
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(np.std(val_max),6)*100
print "Values for C used: "
print [C_SVM[idx] for idx in train_max_arg]

Training for C = 1e-06 		 Time:562.36
Training for C = 3.111111111111111e-06 		 Time:822.92
Training for C = 5.2222222222222226e-06 		 Time:917.54
Training for C = 7.333333333333334e-06 		 Time:983.97
Training for C = 9.444444444444446e-06 		 Time:1058.7
Training for C = 1.1555555555555558e-05 		 Time:1122.16
Training for C = 1.3666666666666669e-05 		 Time:1141.73
Training for C = 1.577777777777778e-05 		 Time:1199.02
Training for C = 1.7888888888888892e-05 		 Time:1221.7
Training for C = 2e-05 		 Time:1259.24
 
Train 	 Validation 	 Validation var 	 C value
99.98 	   69.82 	    0.557 		  1e-06
100.0 	   69.61 	    0.5052 		  3.111111111111111e-06
100.0 	   69.48 	    0.5051 		  5.2222222222222226e-06
100.0 	   69.39 	    0.5188 		  7.333333333333334e-06
100.0 	   69.34 	    0.5339 		  9.444444444444446e-06
100.0 	   69.28 	    0.533 		  1.1555555555555558e-05
100.0 	   69.25 	    0.518 		  1.3666666666666669e-05
100.0 	   69.24 	    0.5316 		  1.577777777777778e-05
100.0 	   69.24 	   

In [5]:
from sklearn import svm
C_SVM = np.linspace(0.1e-6, 10e-6, 20)
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(np.std(val_max),6)*100
print "Values for C used: "
print [C_SVM[idx] for idx in train_max_arg]

Training for C = 1e-07 		 Time:235.84
Training for C = 6.210526315789473e-07 		 Time:450.88
Training for C = 1.1421052631578947e-06 		 Time:562.95
Training for C = 1.663157894736842e-06 		 Time:637.16
Training for C = 2.184210526315789e-06 		 Time:689.71
Training for C = 2.7052631578947364e-06 		 Time:755.39
Training for C = 3.226315789473684e-06 		 Time:780.9
Training for C = 3.7473684210526313e-06 		 Time:817.16
Training for C = 4.2684210526315785e-06 		 Time:849.18
Training for C = 4.789473684210526e-06 		 Time:888.41
Training for C = 5.310526315789473e-06 		 Time:902.35
Training for C = 5.8315789473684205e-06 		 Time:945.27
Training for C = 6.352631578947368e-06 		 Time:948.62
Training for C = 6.873684210526315e-06 		 Time:950.83
Training for C = 7.394736842105263e-06 		 Time:984.33
Training for C = 7.91578947368421e-06 		 Time:1014.18
Training for C = 8.436842105263158e-06 		 Time:1021.95
Training for C = 8.957894736842106e-06 		 Time:1025.93
Training for C = 9.478947368421053e-06

In [6]:
from sklearn import svm
C_SVM = np.linspace(0.1e-6, 2e-6, 10)
#[2e-6] # like in the code from the paper
#[7e-07, 3e-06, 6e-06, 8e-06]
#[1e-08, 5e-08, 1e-07, 5e-07, 1e-06]#, 1e-05, 1e-04]
#[1e-06, 1e-05, 1e-04]
#[1e-06, 5e-06, 1e-05, 5e-05, 1e-04, 5e-04]
#[1e-06, 2e-06, 4e-06, 6e-06, 8e-06, 1e-05, 2e-05]

c_scores_train = []
c_scores_val = []


for c in C_SVM:
    k_class = []
    train_scores = []
    test_scores = []
    print "Training for C = " + str(c),
    st = time.time()
    for k_fold_i, fold in enumerate(k_folds):
        # training the SVM classifier
        fold_ord = fold
        fold_ord.sort()
        fold_ord = list(fold_ord)
        X_train_fold = X_train[fold_ord]#.numpy()
        y_train_fold = y_train[fold_ord]#.numpy()

        clf_i = svm.LinearSVC(C = c)
        #print ('Training fold number ' + str(k_fold_i))
        k_class.append(clf_i.fit(X_train_fold, y_train_fold))
        train_scores.append(clf_i.score(X_train_fold, y_train_fold))
        test_scores.append(clf_i.score(X_val, y_val))
        del clf_i
    nd = time.time()
    print "\t\t Time:" + str(round((nd-st),2))
    c_scores_train.append(train_scores)
    c_scores_val.append(test_scores)
    
print ' '
print "Train \t Validation \t Validation var \t C value"
for num, _ in enumerate(c_scores_val):
    print round(np.mean(c_scores_train[num]),4)*100,
    print "\t  ",
    print round(np.mean(c_scores_val[num]),4)*100,
    print "\t   ",
    print round(np.std(c_scores_val[num]),6)*100,
    print "\t\t ",
    print C_SVM[num] 
    
print ' '
# getting the SVM with the best C
scores_matrix_train = np.zeros((len(C_SVM),10))
scores_matrix_val = np.zeros((len(C_SVM),10))

for num, _ in enumerate(c_scores_val):
    scores_matrix_train[num] = c_scores_train[num]
    scores_matrix_val[num] = c_scores_val[num]

val_max = np.max(scores_matrix_val, axis=0)
train_max_arg = np.argmax(scores_matrix_val, axis=0)

train_max = []

for num, c_idx in enumerate(train_max_arg):
    train_max.append(scores_matrix_train[c_idx][num])

print "Train \t Validation \t Validation var"
print round(np.mean(train_max),4)*100,
print "\t  ",
print round(np.mean(val_max),4)*100,
print "\t   ",
print round(np.std(val_max),6)*100
print "Values for C used: "
print [C_SVM[idx] for idx in train_max_arg]

Training for C = 1e-07 		 Time:255.72
Training for C = 3.1111111111111107e-07 		 Time:363.24
Training for C = 5.222222222222223e-07 		 Time:435.25
Training for C = 7.333333333333333e-07 		 Time:492.06
Training for C = 9.444444444444444e-07 		 Time:541.13
Training for C = 1.1555555555555556e-06 		 Time:582.63
Training for C = 1.3666666666666666e-06 		 Time:623.21
Training for C = 1.5777777777777775e-06 		 Time:654.68
Training for C = 1.788888888888889e-06 		 Time:696.35
Training for C = 2e-06 		 Time:736.05
 
Train 	 Validation 	 Validation var 	 C value
90.07 	   67.73 	    0.463 		  1e-07
98.28 	   69.44 	    0.5001 		  3.1111111111111107e-07
99.77 	   69.74 	    0.5146 		  5.222222222222223e-07
99.95 	   69.8 	    0.5624 		  7.333333333333333e-07
99.98 	   69.79 	    0.5443 		  9.444444444444444e-07
99.99 	   69.81 	    0.5344 		  1.1555555555555556e-06
99.99 	   69.82 	    0.5186 		  1.3666666666666666e-06
100.0 	   69.77 	    0.5402 		  1.5777777777777775e-06
100.0 	   69.79 	    0